In [5]:
import matplotlib.pyplot as plt
import rasterio
import numpy as np
from matplotlib.colors import ListedColormap, BoundaryNorm

In [2]:
resos = [20, 40, 80, 160]
rasters = ['LAI_pine', 'LAI_spruce', 'LAI_decid', 'LAI_tot',
           'canopy_fraction', 'canopy_height', 
          'SOIL_DEPTH', 'twi_dinf', 'twi_d8',
          'dem_clip_no_deps']

for raster in rasters:
    # Choose unit depending on raster type
    if raster.startswith("LAI"):
        unit = "[m² m⁻²]"
    elif raster.startswith("dem"):
        unit = "[m]"
    elif raster == "canopy_fraction":
        unit = "[-]"
    elif raster == "canopy_height":
        unit = "[m]"
    elif raster == 'SOIL_DEPTH':
        unit = "[m]"
    else:
        unit = "[]"  # fallback if new raster types appear
    data_list = []
    
    # Read rasters and store
    for reso in resos:
        file = f'/Users/jpnousu/Krycklan_GIS_data/{reso}m/ALL/{raster}.asc'
        with rasterio.open(file) as src:
            data = src.read(1)
            nodata = src.nodata if src.nodata is not None else -9999
            data = np.where(data == nodata, np.nan, data)
            data_list.append(data)
    
    # Compute global min/max ignoring nodata
    all_values = np.concatenate([d[~np.isnan(d)].ravel() for d in data_list])
    vmin = np.min(all_values)
    vmax = np.max(all_values)
    
    # Define bins for PDF line plots
    num_bins = 20
    bins = np.linspace(vmin, vmax, num_bins + 1)
    bin_centers = (bins[:-1] + bins[1:]) / 2
    
    # Create figure and GridSpec
    fig = plt.figure(figsize=(12, 12))
    gs = fig.add_gridspec(
        nrows=3, ncols=2, 
        height_ratios=[1, 1, 0.5],  # bottom row is half the height
        hspace=0.05, wspace=0.05
    )
    
    # Top 2 rows: 4 LAI maps in 2x2 grid
    for i, reso in enumerate(resos):
        row = i // 2
        col = i % 2
        ax = fig.add_subplot(gs[row, col])
        data = data_list[i]
        file = f'/Users/jpnousu/Krycklan_GIS_data/{reso}m/ALL/{raster}.asc'
        with rasterio.open(file) as src:
            extent = (src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top)
        im = ax.imshow(data, extent=extent, origin='upper', cmap='BrBG',
                       vmin=vmin, vmax=vmax)
        ax.set_title(f'{reso} m')
        ax.axis('off')
    
    # Add horizontal colorbar for maps (spanning both columns)
    #cbar_ax = fig.add_axes([0.2, 0.93, 0.6, 0.015])  # [left, bottom, width, height]
    #fig.colorbar(im, cax=cbar_ax, orientation='horizontal', label='LAI [m2m-2]')
    
    # Vertical colorbar on the left
    cbar_ax = fig.add_axes([0.5, 0.3, 0.01, 0.55])  # [left, bottom, width, height]
    fig.colorbar(im, cax=cbar_ax, orientation='vertical', label=f'{raster} {unit}')
    
    # Bottom row: 1 subplot spanning both columns for line distributions
    ax_dist = fig.add_subplot(gs[2, :])
    for i, reso in enumerate(resos):
        data = data_list[i][~np.isnan(data_list[i])].ravel()
        hist, _ = np.histogram(data, bins=bins, density=True)
        ax_dist.plot(bin_centers, hist, label=f'{reso} m', linewidth=2)
    
    ax_dist.spines['top'].set_visible(False)
    ax_dist.spines['right'].set_visible(False)
    ax_dist.set_xlabel(f'{raster} {unit}')
    ax_dist.set_ylabel('Relative frequency')
    #ax_dist.set_title('LAI distribution')
    ax_dist.legend()
    plt.savefig(f'figs/{raster}_reso.png', dpi=300, bbox_inches="tight")
    plt.close()

In [5]:
resos = [20, 40, 80, 160]
rasters = ['channels', '5haStreams']

#channel_data_list = []

for raster in rasters:
    channel_data_list = []  # reset for each raster
    # Read rasters and preprocess
    for reso in resos:
        file = f'/Users/jpnousu/Krycklan_GIS_data/{reso}m/ALL/{raster}.asc'
        with rasterio.open(file) as src:
            data = src.read(1)
            nodata = src.nodata if src.nodata is not None else -9999
            data = np.where(data == nodata, 0, data)  # nodata → 0
            channel_data_list.append((data, src.bounds))
    
    # Binary colormap and norm
    cmap = ListedColormap(["white", "black"])
    norm = BoundaryNorm([0, 1, 2], cmap.N)  # bins: 0–1, 1–2
    
    # Create figure (2 rows × 2 cols)
    fig, axes = plt.subplots(2, 2, figsize=(10, 10))
    axes = axes.ravel()
    
    for i, reso in enumerate(resos):
        data, bounds = channel_data_list[i]
        extent = (bounds.left, bounds.right, bounds.bottom, bounds.top)
        
        ax = axes[i]
        im = ax.imshow(data, extent=extent, origin="upper", cmap=cmap, norm=norm)
        ax.set_title(f"{reso} m")
        ax.axis("off")
    
    # Vertical discrete colorbar on left
    cbar_ax = fig.add_axes([0.5, 0.4, 0.01, 0.2])
    cbar = fig.colorbar(im, cax=cbar_ax, orientation='vertical')
    cbar.set_ticks([0.25, 1.25])  # centers of 0 and 1 bins
    cbar.set_ticklabels(["No channel", "Channel"])
    #cbar.set_label("Channel presence")
    
    plt.tight_layout()
    plt.savefig(f"figs/{raster}_reso.png", dpi=300, bbox_inches="tight")
    plt.close()

/var/folders/01/f14lxpfj5wl28zjtf5rz8t9c0000gn/T/ipykernel_77781/921998030.py:46: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/var/folders/01/f14lxpfj5wl28zjtf5rz8t9c0000gn/T/ipykernel_77781/921998030.py:46: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


In [6]:
resos = [20, 40, 80, 160]
rasters = ['channels_depth', '5haStreams_depth', 'channels_distance', '5haStreams_distance', 'channels_length', '5haStreams_length']

for raster in rasters:
    channel_data_list = []  # reset for each raster
    
    # Read rasters and preprocess
    for reso in resos:
        file = f'/Users/jpnousu/Krycklan_GIS_data/{reso}m/ALL/{raster}.asc'
        with rasterio.open(file) as src:
            data = src.read(1).astype(float)
            nodata = src.nodata if src.nodata is not None else -9999
            data[data == nodata] = np.nan  # mask nodata as NaN
            channel_data_list.append((data, src.bounds))
        
    # Create figure (2 rows × 2 cols)
    fig, axes = plt.subplots(2, 2, figsize=(10, 10))
    axes = axes.ravel()
    
    # Store last imshow for colorbar
    im = None
    
    for i, reso in enumerate(resos):
        data, bounds = channel_data_list[i]
        extent = (bounds.left, bounds.right, bounds.bottom, bounds.top)
        
        ax = axes[i]
        im = ax.imshow(data, extent=extent, origin="upper", cmap='BrBG')
        ax.set_title(f"{reso} m")
        ax.axis("off")
    
    # Continuous vertical colorbar on left
    cbar_ax = fig.add_axes([0.5, 0.3, 0.01, 0.55])  # [left, bottom, width, height]
    unit = "m"  # <-- change this to your unit
    fig.colorbar(im, cax=cbar_ax, orientation='vertical', label=f'{raster} [{unit}]')
    
    plt.tight_layout()
    plt.savefig(f"figs/{raster}_reso.png", dpi=300, bbox_inches="tight")
    plt.close()

/var/folders/01/f14lxpfj5wl28zjtf5rz8t9c0000gn/T/ipykernel_2840/2547134855.py:37: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


In [69]:
resos = [20, 40, 80, 160]
soil_data_list = []

# Read rasters and preprocess
for reso in resos:
    file = f'/Users/jpnousu/Krycklan_GIS_data/{reso}m/ALL/krycklan_QD_J1_J2_final.asc'
    with rasterio.open(file) as src:
        data = src.read(1)
        nodata = src.nodata if src.nodata is not None else -9999
        data = np.where(data == nodata, np.nan, data)
        soil_data_list.append((data, src.bounds))

# Collect all unique soil class values across resolutions
all_values = np.unique(
    np.concatenate([np.unique(d[~np.isnan(d)]) for d, _ in soil_data_list])
).astype(int)

n_classes = len(all_values)

# Use ColorBrewer Set3 (or any categorical palette)
base_cmap = cm.get_cmap("Set3", max(n_classes, 12))
colors = base_cmap.colors[:n_classes]
cmap = ListedColormap(colors)

# Discrete bins for classes
boundaries = np.append(all_values - 0.5, all_values[-1] + 0.5)
norm = BoundaryNorm(boundaries, ncolors=cmap.N)

# Create figure (2 rows × 2 cols)
fig, axes = plt.subplots(2, 2, figsize=(10, 10))
axes = axes.ravel()

for i, reso in enumerate(resos):
    data, bounds = soil_data_list[i]
    extent = (bounds.left, bounds.right, bounds.bottom, bounds.top)
    
    ax = axes[i]
    im = ax.imshow(data, extent=extent, origin="upper", cmap=cmap, norm=norm)
    ax.set_title(f"{reso} m")
    ax.axis("off")

# Vertical colorbar on the left (like your other figures)
cbar_ax = fig.add_axes([0.5, 0.3, 0.01, 0.55])  # [left, bottom, width, height]
cbar = fig.colorbar(im, cax=cbar_ax, orientation='vertical')
cbar.set_ticks(all_values)
cbar.set_ticklabels([f"{v}" for v in all_values])
cbar.set_label("Soil class [-]")

plt.tight_layout()
plt.savefig("figs/soil_classes_reso.png", dpi=300, bbox_inches="tight")
plt.close()

/var/folders/01/f14lxpfj5wl28zjtf5rz8t9c0000gn/T/ipykernel_68911/2052911096.py:27: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  base_cmap = cm.get_cmap("Set3", max(n_classes, 12))
/var/folders/01/f14lxpfj5wl28zjtf5rz8t9c0000gn/T/ipykernel_68911/2052911096.py:55: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


In [80]:
resos = [20, 40, 80, 160]
data_list = []

# Read rasters
for reso in resos:
    file = f'/Users/jpnousu/Krycklan_GIS_data/{reso}m/ALL/cmask.asc'
    with rasterio.open(file) as src:
        data = src.read(1)
        nodata = src.nodata if src.nodata is not None else -9999
        data = np.where(data == nodata, np.nan, data)
        data_list.append((data, src.bounds))

# Get all unique catchment IDs across resolutions
all_ids = np.unique(
    np.concatenate([np.unique(d[~np.isnan(d)]) for d, _ in data_list])
).astype(int)
n_classes = len(all_ids)

# Discrete colormap: one color per catchment ID
base_cmap = cm.get_cmap("tab20", n_classes)
cmap = ListedColormap(base_cmap(np.arange(n_classes)))

# Norm that maps each unique ID to a color, ignoring missing values
norm = BoundaryNorm(np.arange(n_classes + 1) - 0.5, cmap.N)

# Mapping from ID -> index in colormap
id_to_index = {id_val: i for i, id_val in enumerate(all_ids)}

# Create figure
fig, axes = plt.subplots(2, 2, figsize=(10, 10))
axes = axes.ravel()

for i, reso in enumerate(resos):
    data, bounds = data_list[i]
    # Map catchment IDs to colormap indices
    data_idx = np.full_like(data, np.nan, dtype=float)
    for id_val, idx in id_to_index.items():
        data_idx[data == id_val] = idx
    extent = (bounds.left, bounds.right, bounds.bottom, bounds.top)
    
    ax = axes[i]
    im = ax.imshow(data_idx, extent=extent, origin="upper", cmap=cmap, norm=norm)
    ax.set_title(f"{reso} m")
    ax.axis("off")

# Colorbar on left
cbar_ax = fig.add_axes([0.5, 0.3, 0.01, 0.55])
cbar = fig.colorbar(im, cax=cbar_ax, orientation='vertical')
cbar.set_ticks(np.arange(n_classes))
cbar.set_ticklabels([str(v) for v in all_ids])
cbar.set_label("Catchment ID [-]")

#plt.tight_layout()
plt.savefig("figs/cmask_reso.png", dpi=300, bbox_inches="tight")
plt.close()

/var/folders/01/f14lxpfj5wl28zjtf5rz8t9c0000gn/T/ipykernel_68911/1958587735.py:26: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  base_cmap = cm.get_cmap("tab20", n_classes)
